# Potrzebne importy

In [5]:
from sklearn.metrics import accuracy_score, log_loss, classification_report
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostClassifier
import numpy as np
import pandas as pd
import os

# Tworzenie Database

In [6]:
df=[]
df2=[]
path_elo=['E0-2.csv','E0-3.csv','E0-4.csv','E0-5.csv','E0-6.csv','E0-7.csv',
          'SP1.csv','SP1-2.csv','SP1-3.csv','SP1-4.csv','SP1-5.csv','SP1-6.csv',
          'D1.csv','D1-2.csv','D1-3.csv','D1-4.csv','D1-5.csv','D1-6.csv',
          'F1.csv','F1-2.csv','F1-3.csv','F1-4.csv','F1-5.csv','F1-6.csv',
          'I1.csv','I1-2.csv','I1-3.csv','I1-4.csv','I1-5.csv','I1-6.csv']
path_train=['E0-8.csv','E0-9.csv','E0-10.csv','E0-11.csv','E0-2.csv','E0-3.csv','E0-4.csv','E0-5.csv','E0-6.csv','E0-7.csv','E0-12.csv','E0-13.csv',
            'SP1.csv','SP1-2.csv','SP1-3.csv','SP1-4.csv','SP1-5.csv','SP1-6.csv','SP1-7.csv','SP1-8.csv','SP1-9.csv','SP1-10.csv','SP1-11.csv','SP1-12.csv',
            'D1.csv','D1-2.csv','D1-3.csv','D1-4.csv','D1-5.csv','D1-6.csv','D1-7.csv','D1-8.csv','D1-9.csv','D1-10.csv','D1-11.csv','D1-12.csv',
            'F1.csv','F1-2.csv','F1-3.csv','F1-4.csv','F1-5.csv','F1-6.csv','F1-7.csv','F1-8.csv','F1-9.csv','F1-10.csv','F1-11.csv','F1-12.csv',
            'I1.csv','I1-2.csv','I1-3.csv','I1-4.csv','I1-5.csv','I1-6.csv','I1-7.csv','I1-8.csv','I1-9.csv','I1-10.csv','I1-11.csv','I1-12.csv']
for i in path_elo:
    s=os.path.join('elo',i)
    dfs=pd.read_csv(s)
    df.append(dfs)
for i in path_train:
    s=os.path.join('train',i)
    dfs=pd.read_csv(s)
    df2.append(dfs)
elo=pd.concat(df)
data=pd.concat(df2)

# Czyszczenie danych

In [7]:
data['Date'] = pd.to_datetime(data['Date'], format='mixed',dayfirst=True,errors='coerce')
data=data.sort_values(by='Date',ascending=True)
data=pd.DataFrame(data,columns=['Date','HomeTeam','AwayTeam','FTHG','FTAG','FTR'])
data=data.dropna()
data=data.reset_index()
data=data.replace({'FTR':{'H':2,'D':1,'A':0}})
data=pd.DataFrame(data,columns=['Date','HomeTeam','AwayTeam','FTHG','FTAG','FTR'])

elo['Date'] = pd.to_datetime(elo['Date'], format='mixed',dayfirst=True,errors='coerce')
elo=elo.sort_values(by='Date',ascending=True)
elo=pd.DataFrame(elo,columns=['Date','HomeTeam','AwayTeam','FTHG','FTAG','FTR'])
elo=elo.dropna()
elo=elo.reset_index()
elo=elo.replace({'FTR':{'H':2,'D':1,'A':0}})
elo=pd.DataFrame(elo,columns=['Date','HomeTeam','AwayTeam','FTHG','FTAG','FTR'])

/var/folders/lq/rknhdtpd1yjd6_2c3_5hm6sm0000gn/T/ipykernel_12270/1992878739.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data=data.replace({'FTR':{'H':2,'D':1,'A':0}})
/var/folders/lq/rknhdtpd1yjd6_2c3_5hm6sm0000gn/T/ipykernel_12270/1992878739.py:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  elo=elo.replace({'FTR':{'H':2,'D':1,'A':0}})


# Tworzenie rankingu ELO

In [8]:
rating=pd.DataFrame(columns=['Team','Rating'])
x = pd.concat([data['HomeTeam'],elo['HomeTeam']])
rating['Team'] = x.unique()
rating['Rating']=1500.0

for index, row in elo.iterrows():
    home = row['HomeTeam']
    away = row['AwayTeam']
    rh = rating.loc[rating['Team'] == home, "Rating"].iloc[0]
    ra = rating.loc[rating['Team'] == away, "Rating"].iloc[0]
    dr = np.absolute(rh-ra)
    we = 1/(np.power(10,(-dr/400))+1)
    n = np.abs(row['FTHG'] - row['FTAG'])
    if n==1:
        g=1
    if n==2:
        g=1.5
    if n>2:
        g=(11+n)/8
    k=10
    if row['FTR'] == 1:
        p = k * (0.5 - we)
        rating.loc[rating['Team'] == home, "Rating"] = rh+ p
        rating.loc[rating['Team'] == away, "Rating"] = ra +p
    if row['FTR'] == 0:
        p = k * g * (0 - we)
        rating.loc[rating['Team'] == home, "Rating"] =rh+ p
        rating.loc[rating['Team'] == away, "Rating"] =ra- p
    if row['FTR'] == 2:
        p = k * g * (1 - we)
        rating.loc[rating['Team'] == home, "Rating"] =rh+ p
        rating.loc[rating['Team'] == away, "Rating"] =ra- p

HELO = []
AELO = []
for index, row in data.iterrows():
    home = row['HomeTeam']
    away = row['AwayTeam']
    rh = rating.loc[rating['Team'] == home, "Rating"].iloc[0]
    ra = rating.loc[rating['Team'] == away, "Rating"].iloc[0]
    HELO.append(rh)
    AELO.append(ra)
    dr = np.absolute(rh-ra)
    we = 1/(np.power(10,(-dr/400))+1)
    n = np.abs(row['FTHG'] - row['FTAG'])
    if n==1:
        g=1
    if n==2:
        g=1.5
    if n>2:
        g=(11+n)/8
    k=10
    if row['FTR'] == 1:
        p = k * (0.5 - we)
        rating.loc[rating['Team'] == home, "Rating"] = rh+ p
        rating.loc[rating['Team'] == away, "Rating"] = ra +p
    if row['FTR'] == 0:
        p = k * g * (0 - we)
        rating.loc[rating['Team'] == home, "Rating"] =rh+ p
        rating.loc[rating['Team'] == away, "Rating"] =ra- p
    if row['FTR'] == 2:
        p = k * g * (1 - we)
        rating.loc[rating['Team'] == home, "Rating"] =rh+ p
        rating.loc[rating['Team'] == away, "Rating"] =ra- p
rating = rating.sort_values(by='Rating',ascending=False)
data['HELO'] = HELO
data['AELO'] = AELO

rating.head()

,Team,Rating
51,Barcelona,3754.771669
1,Bayern Munich,3668.435390
49,Real Madrid,3646.131962
48,Paris SG,3311.606911
57,Man City,3281.296977


# Tworzenie parametrów

In [9]:
# Ostatnie 3 meczy HTHGL
data['prev_HTHG3']=data.groupby('HomeTeam')['FTHG'].rolling(window=3,min_periods=1).mean().reset_index(level=0,drop=True)
data['HTHG3']=data.groupby('HomeTeam')['prev_HTHG3'].shift(1).fillna(0)
# Ostatnie 3 meczy HTHGl
data['prev_HTHGL3']=data.groupby('HomeTeam')['FTAG'].rolling(window=3,min_periods=1).mean().reset_index(level=0,drop=True)
data['HTHGL3']=data.groupby('HomeTeam')['prev_HTHGL3'].shift(1).fillna(0)
# Ostatnie 3 mecze ATAG
data['prev_ATAG3']=data.groupby('AwayTeam')['FTAG'].rolling(window=3,min_periods=1).mean().reset_index(level=0,drop=True)
data['ATAG3']=data.groupby('AwayTeam')['prev_ATAG3'].shift(1).fillna(0)
# Ostatnie 3 mecze ATAGL
data['prev_ATAGL3']=data.groupby('AwayTeam')['FTHG'].rolling(window=3,min_periods=1).mean().reset_index(level=0,drop=True)
data['ATAGL3']=data.groupby('AwayTeam')['prev_ATAGL3'].shift(1).fillna(0)
# Ostatnie 20 meczy HTHGL
data['prev_HTHG20']=data.groupby('HomeTeam')['FTHG'].rolling(window=20,min_periods=1).mean().reset_index(level=0,drop=True)
data['HTHG20']=data.groupby('HomeTeam')['prev_HTHG20'].shift(1).fillna(0)
# Ostatnie 20 meczy HTHGl
data['prev_HTHGL20']=data.groupby('HomeTeam')['FTAG'].rolling(window=20,min_periods=1).mean().reset_index(level=0,drop=True)
data['HTHGL20']=data.groupby('HomeTeam')['prev_HTHGL20'].shift(1).fillna(0)
# Ostatnie 20 mecze ATAG
data['prev_ATAG20']=data.groupby('AwayTeam')['FTAG'].rolling(window=20,min_periods=1).mean().reset_index(level=0,drop=True)
data['ATAG20']=data.groupby('AwayTeam')['prev_ATAG20'].shift(1).fillna(0)
# Ostatnie 20 mecze ATAGL
data['prev_ATAGL20']=data.groupby('AwayTeam')['FTHG'].rolling(window=20,min_periods=1).mean().reset_index(level=0,drop=True)
data['ATAGL20']=data.groupby('AwayTeam')['prev_ATAGL20'].shift(1).fillna(0)
#procent wygranych i przegranych dla 20 meczy
data['pHW']=(data['FTR']==2).astype(int)
data['pHL']=(data['FTR']==0).astype(int)
data['pAW']=(data['FTR']==2).astype(int)
data['pAL']=(data['FTR']==0).astype(int)
data['HW20a']=data.groupby('HomeTeam')['pHW'].rolling(window=20,min_periods=1).mean().reset_index(level=0,drop=True)
data['HL20a']=data.groupby('HomeTeam')['pHL'].rolling(window=20,min_periods=1).mean().reset_index(level=0,drop=True)
data['AW20a']=data.groupby('AwayTeam')['pAW'].rolling(window=20,min_periods=1).mean().reset_index(level=0,drop=True)
data['AL20a']=data.groupby('AwayTeam')['pAL'].rolling(window=20,min_periods=1).mean().reset_index(level=0,drop=True)
data['HW20']=data.groupby('HomeTeam')['HW20a'].shift(1).fillna(0)
data['HL20']=data.groupby('HomeTeam')['HL20a'].shift(1).fillna(0)
data['AW20']=data.groupby('AwayTeam')['AW20a'].shift(1).fillna(0)
data['AL20']=data.groupby('AwayTeam')['AL20a'].shift(1).fillna(0)
# dla 3 ostatnich meczy
data['HW3a']=data.groupby('HomeTeam')['pHW'].rolling(window=3,min_periods=1).mean().reset_index(level=0,drop=True)
data['HL3a']=data.groupby('HomeTeam')['pHL'].rolling(window=3,min_periods=1).mean().reset_index(level=0,drop=True)
data['AW3a']=data.groupby('AwayTeam')['pAW'].rolling(window=3,min_periods=1).mean().reset_index(level=0,drop=True)
data['AL3a']=data.groupby('AwayTeam')['pAL'].rolling(window=3,min_periods=1).mean().reset_index(level=0,drop=True)
data['HW3']=data.groupby('HomeTeam')['HW3a'].shift(1).fillna(0)
data['HL3']=data.groupby('HomeTeam')['HL3a'].shift(1).fillna(0)
data['AW3']=data.groupby('AwayTeam')['AW3a'].shift(1).fillna(0)
data['AL3']=data.groupby('AwayTeam')['AL3a'].shift(1).fillna(0)

# Dane dla druzyny gospodarzy jako goście
HTAG = []
HTAG3= []
HTAW =[]
HTAW3 = []
HTAL =[]
HTAL3 = []
for i in range(len(data)):
    current_sum = 0
    ile=0
    current_sum3 = 0
    ile3=0
    w=0
    w3 =0
    l=0
    l3=0
    wartosc_kryterium_b = data['HomeTeam'][i]
    j2=i
    while(ile<20 and j2>0):
        j2-=1
        if data.loc[j2,'AwayTeam'] == wartosc_kryterium_b:
            ile+=1
            current_sum += data.loc[j2, 'FTAG']
            if data.loc[j2,'FTR'] == 0:
                w+=1
            if data.loc[j2,'FTR'] == 2:
                l+=1
    j =i
    while (ile3<3 and j>0):
        j-=1
        if data.loc[j,'AwayTeam'] == wartosc_kryterium_b:
            if data.loc[j,'FTR'] == 0:
                w3+=1
            if data.loc[j,'FTR'] == 2:
                l3+=1
            ile3+=1
            current_sum3 += data.loc[j, 'FTAG']
    if ile==0:
            ile=1
    if ile3==0:
            ile3=1
        
    HTAG.append(current_sum / ile)
    HTAG3.append(current_sum3 / ile3)
    HTAW.append(w/ile)
    HTAL.append(l/ile)
    HTAW3.append(w3/ile3)
    HTAL3.append(l3/ile3)
        
data['HTAG'] = HTAG
data['HTAG5'] = HTAG3
data['HTAW'] = HTAW
data['HTAL']= HTAL
data['HTAW3'] = HTAW3
data['HTAL3'] = HTAL3

HTAGL = []
HTAGL3 = []
sumy_powyzej = []
ile_powyzej = []
for i in range(len(data)):
        current_sum = 0
        ile=0
        current_sum3 =0
        ile3 =0
        wartosc_kryterium_b = data['HomeTeam'][i]
        j2=i
        while(ile<20 and j2>0):
            j2-=1
            if data.loc[j2,'AwayTeam'] == wartosc_kryterium_b:
                ile+=1
                current_sum += data.loc[j2, 'FTHG']
        j =i
        while (ile3<3 and j>0):
            j-=1
            if data.loc[j,'AwayTeam'] == wartosc_kryterium_b:
                ile3+=1
                current_sum3 += data.loc[j, 'FTHG']
        if ile==0:
            ile=1
        if ile3==0:
            ile3=1
        HTAGL.append(current_sum / ile)
        HTAGL3.append(current_sum3 / ile3)
data['HTAGL'] = HTAGL
data['HTAGL3'] = HTAGL3

# Dane dla druzyny gosci jako gospodarze

ATHG = []
ATHG3 = []
ATHW = []
ATHW3 = []
ATHL =[]
ATHL3 =[]
for i in range(len(data)):
        current_sum = 0
        ile=0
        current_sum3 =0
        ile3=0
        w=0
        w3 =0
        l=0
        l3=0
        wartosc_kryterium_b = data['AwayTeam'][i]
        j2=i
        while(ile<20 and j2>0):
            j2-=1
            if data.loc[j2,'HomeTeam'] == wartosc_kryterium_b:
                ile+=1
                current_sum += data.loc[j2, 'FTHG']
                if data.loc[j2,'FTR'] == 2:
                    w+=1
                if data.loc[j2,'FTR'] == 0:
                    l+=1
        j=i
        while (ile3<3 and j>0):
            j-=1
            if data.loc[j,'HomeTeam'] == wartosc_kryterium_b:
                ile3+=1
                current_sum3 += data.loc[j, 'FTHG']
                if data.loc[j,'FTR'] == 2:
                    w3+=1
                if data.loc[j,'FTR'] == 0:
                    l3+=1
        if ile==0:
            ile=1
        if ile3==0:
            ile3=1
        ATHG.append(current_sum / ile)
        ATHG3.append(current_sum3 / ile3)
        ATHW.append(w/ile)
        ATHL.append(l/ile)
        ATHW3.append(w3/ile3)
        ATHL3.append(l3/ile3)
data['ATHG'] = ATHG
data['ATHG3'] = ATHG3
data['ATHW'] = ATHW
data['ATHW3'] = ATHW3
data['ATHL'] = ATHL
data['ATHL3'] = ATHL3
ATHGL =[]
ATHGL3 =[]
for i in range(len(data)):
        current_sum = 0
        ile=0
        current_sum3=0
        ile3=0
        wartosc_kryterium_b = data['AwayTeam'][i]
        j2=i
        while(ile<20 and j2>0):
            j2-=1
            if data.loc[j2,'HomeTeam'] == wartosc_kryterium_b:
                ile+=1
                current_sum += data.loc[j2, 'FTAG']
        j=i
        while (ile3<3 and j>0):
            j-=1
            if data.loc[j,'HomeTeam'] == wartosc_kryterium_b:
                ile3+=1
                current_sum3 += data.loc[j, 'FTAG']
        if ile==0:
            ile=1
        if ile3==0:
            ile3=1
        ATHGL.append(current_sum / ile)
        ATHGL3.append(current_sum3 / ile3)
data['ATHGL'] = ATHGL
data['ATHGL3'] = ATHGL3
data=data.drop(columns=['prev_HTHGL20','prev_HTHG20','prev_ATAG20','prev_ATAGL20','prev_HTHGL3','prev_HTHG3','prev_ATAG3','prev_ATAGL3','pHW','pHL','pAL','pAW','HW20a','HL20a','AL20a','AW3a','HW3a','HL3a','AL3a','AW3a'])

# Train-test split

In [12]:
split_date = pd.to_datetime('2023-07-01')
data_train = data[data['Date'] <= split_date]
data_test = data[data['Date'] > split_date]

y_train=pd.DataFrame(data_train,columns=['FTR'])
y_test=pd.DataFrame(data_test,columns=['FTR'])
X_train=data_train.drop(columns=['Date','FTHG','FTAG','FTR'])
X_test=data_test.drop(columns=['Date','FTHG','FTAG','FTR'])

# Nauka Modelu

In [13]:
cat_features = ['HomeTeam', 'AwayTeam']


model = CatBoostClassifier(
    loss_function='MultiClass',
    verbose=0,
    cat_features=cat_features,
    random_state=42
)
param_grid={'iterations': [200, 500, 1000], 'learning_rate': [0.01, 0.03, 0.05], 'depth': [4, 6, 8] }

grid = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='accuracy',
    cv=3,
    n_jobs=-1
)

grid.fit(X_train, y_train)

GridSearchCV(cv=3,
             estimator=<catboost.core.CatBoostClassifier object at 0x178484860>,
             n_jobs=-1,
             param_grid={'depth': [4, 6, 8], 'iterations': [200, 500, 1000],
                         'learning_rate': [0.01, 0.03, 0.05]},
             scoring='accuracy')

# Wyniki modelu

In [14]:
print("Najlepszy wynik",grid.best_score_)
print("Accuracy score:",accuracy_score(y_test,grid.best_estimator_.predict(X_test)))
print("Log loss:",log_loss(y_test,grid.best_estimator_.predict_proba(X_test)))
print(classification_report(y_test,grid.predict(X_test)))


Najlepszy wynik 0.6826367090698827
Accuracy score: 0.677796803652968
Log loss: 0.6906167400152685
              precision    recall  f1-score   support

           0       0.60      0.62      0.61      1113
           1       0.53      0.33      0.41       900
           2       0.77      0.93      0.84      1491

    accuracy                           0.68      3504
   macro avg       0.64      0.63      0.62      3504
weighted avg       0.66      0.68      0.66      3504

